In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/datos-precios-viviendas-y-mas-espaa/datos_unidos_meses.csv')
df_edad = pd.read_csv('/kaggle/input/datos-precios-viviendas-y-mas-espaa/df_mineria_datos_edades.csv')
df_vivienda = pd.read_csv('/kaggle/input/datos-precios-viviendas-y-mas-espaa/df_mineria_datos_vivienda.csv')#le doy una variable pero para este estudio no voy a utilizar
#esta parte del dataset ya que da datos geograficos

In [ ]:
print('INFORAMCION DE CADA DATASET A UTILIZAR Y PRIMERAS LINEAS')

print('datos unidos:')
print(df.info())
print(df.head())

print('datos edades:')
print(df_edad.info())
print(df_edad.head())

print(df.columns)


In [ ]:
#Editamos el dataset para hacerlo mas comodo para su uso

#revisamos elementos nulos y que no haya duplicados

print("\nMissing Values in Each Column:")
print(df.isnull().sum())


print("\nDuplicate Rows:", df.duplicated().sum())
df.drop_duplicates(inplace=True)


#convertimos el formato de la columan year a uno de date y ponemos 2 columnas con mes y año
df['year'] = pd.to_datetime(df['year'])
df['anho'] = df['year'].dt.year
df['month'] = df['year'].dt.month

#hacemos una nueva columna con el precio medio de todos los materiales para mas comodidad

df['precio_mats'] = df[['precio_Acero','precio_Aluminio','precio_Cemento','precio_Ceramica','precio_Cobre','precio_Energía','precio_Ligantes','precio_Madera']].mean(axis=1)



In [ ]:
#comprobamos de nuevo el dataset y tambien revisamos cada columna
print('dataset editado')
print(df.info())
print(df.head())



In [ ]:
#comprobaciones estadisticas basicas

print('Summary statistics:')
print(df.describe())

# Comparacion de varios datos segun comunidad 

In [ ]:
var1 = ['precio', 'IPC', 'edad_media', 'num_hipotecas', 'num_inmigracion', 'precio_mats']

fig, axes = plt.subplots(3, 2, figsize=(12, 4*3))

for i, var in enumerate(var1):
    row, col = divmod(i, 2)
    sns.boxplot(x='comunidad', y=var, data=df, ax=axes[row, col])
    axes[row, col].set_xticklabels(axes[row, col].get_xticklabels(), rotation=45, ha='right')

for i in range(len(var1), len(axes)):
    fig.delaxes(axes[i])

fig.suptitle('Distribución de  variables segun Comunidad', fontweight='bold', fontsize=15)
plt.tight_layout()
plt.show()

Por lo que se puede observar tanto el ipc como los precios de los materiales no cambian mucho de una comunidad a otra mientras que la edad varia bastante,el precio segun se observa
es bastante constante en la mayoria de comunidades salvo en 4 donde se dispara:pais vasco,islas canarias y baleares,madrid y cataluña.Por ultimo podemos observar que la inmigracion es muy desigual
estando la gran mayoria en la comunidad valenciana

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

var1 = ['precio', 'IPC', 'edad_media', 'num_hipotecas', 'num_inmigracion','precio_mats']

fig, axes = plt.subplots(2, 3, figsize=(16, 10), sharex=True, sharey=False)

axes = axes.flatten()

scatter = sns.scatterplot(x='anho', y=var1[0], hue='comunidad', data=df, s=50, alpha=0.8, ax=axes[0])

handles, labels = axes[0].get_legend_handles_labels()
axes[0].legend_.remove() 

for i, var in enumerate(var1):
    ax = axes[i]
    sns.scatterplot(x='anho', y=var, hue='comunidad', data=df, s=50, alpha=0.8, ax=ax, legend=False)
    sns.regplot(x='anho', y=var, data=df, order=3, scatter=False, color='black', line_kws={'linewidth': 2}, ax=ax)
    
    ax.set_title(f"{var}")
    ax.set_xlabel("Año")
    ax.set_ylabel(var)

for i in range(len(var1), len(axes)):
    fig.delaxes(axes[i])

fig.legend(handles, labels, title="Comunidad", bbox_to_anchor=(1.02, 0.5), loc='center left')

plt.suptitle("Evolución  durante los años por comunidades", fontsize=14)
plt.tight_layout(rect=[0, 0, 0.85, 1])  
plt.show()

En estas graficas podemos observar como sobretodo en el 2010 hubo un aumento drastico tanto del precio coomo de la inmigracion aunque en los años sigueintes se normalizo,sin embargo el ipc y los precios
de los materiales no han dejado de subir en ningun momento
El numero de hipotecas tambein fue muy acusado entre 2005 y 2010 pero desde ahi ha bajado de forma drastica.
Ademas la edad media tamebin ha subido en todas las comunidades de forma constante aunque no de forma tan brusca como el ipc

In [ ]:
#mapa de correlacion


plt.figure(figsize=(10, 8))
num = df[['codigo_comunidad','precio', 'IPC', 'edad_media', 'num_hipotecas', 'num_inmigracion','precio_mats']].corr()
sns.heatmap(num, annot=True, cmap='coolwarm',fmt=".2f")
plt.title('Correlation Heatmap of Numeric Features')
plt.tight_layout()
plt.show()




# MODELO DE REGRESION LINEAL

In [ ]:
#modelo predictivo

# Predictive Modeling for Income based on Key Variables
# Selecting relevant features
features = ['codigo_comunidad','IPC', 'edad_media', 'num_hipotecas', 'num_inmigracion','precio_mats']
target = 'precio'

df_filtered = df[features + [target]].dropna()  # Remove missing values

X = df_filtered[features]
y = df_filtered[target]

# Adding a constant for the intercept
X = sm.add_constant(X)

In [ ]:
#modelo de regresion lineal

model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
#modelo de regresion lineal con entreno de variables

# Predictive Modeling: Predicting Housing Prices

# For our simple linear regression, we select a set of predictive features.
# We remove non-numeric and identifier features.
features = ['codigo_comunidad','IPC','edad_media', 'mano_obra','num_hipotecas', 'num_inmigracion','precio_mats']

# Only select rows that have no missing values in the features and target columns
df_model = df.dropna(subset=features + ['precio'])

X = df_model[features]
y = df_model['precio']

# Split into training and testing sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print('Linear Regression Performance:')
print('R2 Score:', r2)
print('Mean Absolute Error:', mae)
print('Mean Squared Error:', mse)

# Optionally, plot a scatter plot of predicted vs actual values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.7, color='orange')
plt.xlabel('Actual Prices')
plt.ylabel('Predicted Prices')
plt.title('Actual vs Predicted Housing Prices')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='blue')
plt.tight_layout()
plt.show()

In [ ]:
# Define the polynomial degree
degree = 2  # Quadratic model

# Selecting the features for the polynomial regression
features_pol = ['codigo_comunidad','IPC','edad_media', 'mano_obra','num_hipotecas', 'num_inmigracion','precio_mats']
target_pol = 'precio'

df_filtered_pol = df[features_pol + [target_pol]].dropna()

X_pol = df_filtered_pol[features_pol]
y_pol = df_filtered_pol[target_pol]


# Create a polynomial regression model
pol_model = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Fit the model
pol_model.fit(X_pol, y_pol)

# Predict values
y_pol_pred = pol_model.predict(X_pol)

# Scatter plot of actual vs predicted income
plt.figure(figsize=(8, 5))
plt.scatter(y_pol, y_pol_pred, alpha=0.3, color="red")
plt.xlabel("Precio actual")
plt.ylabel("prevision del precio")
plt.title(f"Polynomial Regression (Degree {degree}) - Actual vs Predicted Income")
plt.plot([y_pol.min(), y_pol.max()], [y_pol.min(), y_pol.max()], color='blue')
plt.show()

# Model score (R-squared value)
pol_r2_score = pol_model.score(X_pol, y_pol)
print(f"Polynomial Regression R² Score (Degree {degree}): {pol_r2_score:.4f}")

# COMPARACION DE VARIOS ESTUDIOS A FUTURO DEL PRECIO

In [ ]:
#comparacion de varios estudios
#vovlemos a crear una regresion polinomica sin grafica esta vez

degree = 2

features = ['codigo_comunidad','IPC','edad_media', 'mano_obra','num_hipotecas', 'num_inmigracion','precio_mats']
target = 'precio'

df_filter = df[features + [target]].dropna()

X = df_filter[features]
y = df_filter[target]




#creamos un modelo de regresion polinomica
pol_model = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Fit the model
pol_model.fit(X, y)
pol_r2 = pol_model.score(X, y)

In [ ]:
# Random Forest Model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X, y)
rf_r2 = rf_model.score(X, y)


In [ ]:
# Gradient Boosting Model
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
gb_model.fit(X, y)
gb_r2 = gb_model.score(X, y)

In [ ]:
# Feature Importance Analysis
rf_feature_importance = rf_model.feature_importances_
gb_feature_importance = gb_model.feature_importances_
# Convert feature importances into DataFrame
rf_importance_df = pd.DataFrame({"Feature": X.columns, "Importance": rf_feature_importance})
gb_importance_df = pd.DataFrame({"Feature": X.columns, "Importance": gb_feature_importance})

# Sort feature importances
rf_importance_df = rf_importance_df.sort_values(by="Importance", ascending=False)
gb_importance_df = gb_importance_df.sort_values(by="Importance", ascending=False)

# Print R² scores
print(f"Polynomial Regression R² Score: {pol_r2:.4f}")
print(f"Random Forest R² Score: {rf_r2:.4f}")
print(f"Gradient Boosting R² Score: {gb_r2:.4f}")

In [ ]:
# Plot Feature Importance - Random Forest
plt.figure(figsize=(10, 5))
sns.barplot(x=rf_importance_df["Importance"], y=rf_importance_df["Feature"])
plt.title("Feature Importance - Random Forest")
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.show()

In [ ]:
# Plot Feature Importance - Gradient Boosting
plt.figure(figsize=(10, 5))
sns.barplot(x=gb_importance_df["Importance"], y=gb_importance_df["Feature"])
plt.title("Feature Importance - Gradient Boosting")
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.show()

In [ ]:
# Split the dataset into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define hyperparameter grid for Random Forest
rf_param_grid = {
    "n_estimators": [50, 100, 200],  # Reduce the number of trees
    "max_depth": [10, 20, None],  # Limit tree depth to prevent overfitting
    "min_samples_split": [2, 5, 10],  # Minimum samples required to split a node
    "min_samples_leaf": [1, 2, 4],  # Minimum samples required at a leaf node
}

# Define hyperparameter grid for Gradient Boosting
gb_param_grid = {
    "n_estimators": [50, 100, 200],  # Number of boosting stages
    "learning_rate": [0.01, 0.1, 0.2],  # Lower learning rate for better generalization
    "max_depth": [3, 5, 10],  # Depth of individual trees
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}

In [ ]:
# Grid Search for Random Forest
rf_grid_search = GridSearchCV(RandomForestRegressor(random_state=42), rf_param_grid, cv=3, n_jobs=-1, scoring="r2")
rf_grid_search.fit(X_train, y_train)

# Grid Search for Gradient Boosting
gb_grid_search = GridSearchCV(GradientBoostingRegressor(random_state=42), gb_param_grid, cv=3, n_jobs=-1, scoring="r2")
gb_grid_search.fit(X_train, y_train)

# Best parameters found
best_rf_model = rf_grid_search.best_estimator_
best_gb_model = gb_grid_search.best_estimator_

print(f"Best Random Forest Parameters: {rf_grid_search.best_params_}")
print(f"Best Gradient Boosting Parameters: {gb_grid_search.best_params_}")

In [ ]:
# Evaluate on the test set
rf_r2_tuned = best_rf_model.score(X_test, y_test)
gb_r2_tuned = best_gb_model.score(X_test, y_test)

print(f"Tuned Random Forest R² Score (Test Set): {rf_r2_tuned:.4f}")
print(f"Tuned Gradient Boosting R² Score (Test Set): {gb_r2_tuned:.4f}")

In [ ]:
# Define improved hyperparameter grid for Random Forest
rf_param_grid = {
    "n_estimators": [50, 100],  # Reduce number of trees
    "max_depth": [5, 10],  # Force shallower trees
    "min_samples_split": [10, 20],  # Require more samples per split
    "min_samples_leaf": [4, 8],  # Require more samples per leaf
    "max_features": ["sqrt", "log2"],  # Reduce number of features per tree
}

# Define improved hyperparameter grid for Gradient Boosting
gb_param_grid = {
    "n_estimators": [50, 100],  # Reduce number of boosting stages
    "learning_rate": [0.01, 0.05],  # Lower learning rate for better generalization
    "max_depth": [3, 5],  # Force shallower trees
    "min_samples_split": [10, 20],  # Require more samples per split
    "min_samples_leaf": [4, 8],  # Require more samples per leaf
    "subsample": [0.7, 0.85],  # Randomly sample training data to reduce overfitting
}

In [ ]:
# Grid Search for Random Forest (more regularized)
rf_grid_search = GridSearchCV(RandomForestRegressor(random_state=42), rf_param_grid, cv=3, n_jobs=-1, scoring="r2")
rf_grid_search.fit(X_train, y_train)

# Grid Search for Gradient Boosting (more regularized)
gb_grid_search = GridSearchCV(GradientBoostingRegressor(random_state=42), gb_param_grid, cv=3, n_jobs=-1, scoring="r2")
gb_grid_search.fit(X_train, y_train)

# Best models
best_rf_model = rf_grid_search.best_estimator_
best_gb_model = gb_grid_search.best_estimator_

print(f"Best Random Forest Parameters: {rf_grid_search.best_params_}")
print(f"Best Gradient Boosting Parameters: {gb_grid_search.best_params_}")

In [ ]:
# Evaluate on the test set
rf_r2_tuned = best_rf_model.score(X_test, y_test)
gb_r2_tuned = best_gb_model.score(X_test, y_test)

print(f"Tuned (Regularized) Random Forest R² Score (Test Set): {rf_r2_tuned:.4f}")
print(f"Tuned (Regularized) Gradient Boosting R² Score (Test Set): {gb_r2_tuned:.4f}")

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score

# Define hyperparameters for Ridge and Lasso Regression
alpha_values = [0.01, 0.1, 1, 10, 100]  # Strength of regularization

# Ridge Regression (L2 Regularization)
ridge_scores = []
for alpha in alpha_values:
    ridge_model = Ridge(alpha=alpha)
    ridge_model.fit(X_train, y_train)
    score = ridge_model.score(X_test, y_test)
    ridge_scores.append((alpha, score))

# Lasso Regression (L1 Regularization)
lasso_scores = []
for alpha in alpha_values:
    lasso_model = Lasso(alpha=alpha, max_iter=10000)
    lasso_model.fit(X_train, y_train)
    score = lasso_model.score(X_test, y_test)
    lasso_scores.append((alpha, score))

# Print results
print("Ridge Regression R² Scores:")
for alpha, score in ridge_scores:
    print(f"Alpha {alpha}: R² = {score:.4f}")

print("\nLasso Regression R² Scores:")
for alpha, score in lasso_scores:
    print(f"Alpha {alpha}: R² = {score:.4f}")

In [ ]:
from keras.models import Sequential
from keras.layers import Input, Dense
# Define features and target variable
features = ['codigo_comunidad','IPC','edad_media', 'mano_obra','num_hipotecas', 'num_inmigracion','precio_mats']
target = 'precio'


df_filtered = df[features + [target]].dropna()  # Remove missing values
X = df_filtered[features]
y = df_filtered[target]

# Split the dataset into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features (important for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the Neural Network model
model = Sequential([
    Input(shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),  # Hidden layer 1
    Dense(32, activation='relu'),  # Hidden layer 2
    Dense(1)  # Output layer (regression task, so no activation function)
])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



# Define a neural network
modeltensor = Sequential([
    Input(shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)  # salida para regresión
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test), verbose=1)

In [ ]:
# Evaluate the model
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Neural Network Test MAE: {mae:.4f}")

In [ ]:
# Predict on test set
y_pred = model.predict(X_test_scaled)

In [ ]:
# Compute R² Score
from sklearn.metrics import r2_score
r2_nn = r2_score(y_test, y_pred)
print(f"Neural Network R² Score: {r2_nn:.4f}")

In [ ]:
# Save the trained model
model.save("income_prediction_model.h5")
print("Model saved successfully.")

In [ ]:
#resultados de los diferentes modelos

print('\nregresion polinomica:')

print(f"Polynomial Regression R² Score: {pol_r2:.4f}")

print('\ngradiente y bosque aleatorio:')

print(f"Tuned (Regularized) Random Forest R² Score (Test Set): {rf_r2_tuned:.4f}")
print(f"Tuned (Regularized) Gradient Boosting R² Score (Test Set): {gb_r2_tuned:.4f}")

print('\nridge y lasso:')

print("Ridge Regression R² Scores:")
for alpha, score in ridge_scores:
    print(f"Alpha {alpha}: R² = {score:.4f}")

print("\nLasso Regression R² Scores:")
for alpha, score in lasso_scores:
    print(f"Alpha {alpha}: R² = {score:.4f}")

print('\nred neuronal:')

print(f"Neural Network R² Score: {r2_nn:.4f}")

Gradient Boosting y Random Forest: Estos modelos son altamente efectivos, con R² superiores a 0.90, y deberían ser considerados como las opciones principales para la predicción de precios en este caso.

Regresión Polinómica: Aunque este modelo no supera a los modelos de ensemble, tiene un buen rendimiento (R² = 0.7478) y puede ser útil si se busca una solución más sencilla y fácil de interpretar, aunque con un rendimiento algo inferior.

Modelos Lineales (Ridge y Lasso): No aportan mejoras significativas en comparación con otros modelos. Ridge y Lasso con los diferentes valores de alpha no logran un buen ajuste (siendo el R² consistentemente bajo), lo que podría indicar que una regularización más fuerte no es beneficiosa en este caso.

Red Neuronal: Aunque tiene un rendimiento moderado (R² = 0.6091), está por debajo de los modelos de ensemble. Esto sugiere que este conjunto de datos no es el más adecuado para una red neuronal, o que la red necesita ajustes adicionales (como más capas, diferentes funciones de activación o mayor cantidad de datos).